In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import linear_model, metrics
from sklearn.metrics import (mean_squared_error, r2_score, mean_absolute_error, 
mean_squared_log_error, explained_variance_score, max_error)
from sklearn.model_selection import LeaveOneOut, cross_val_score, cross_val_predict
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor

# Load training and testing data
X_train = np.loadtxt("X_train.csv", delimiter=',', skiprows=1)
X_test = np.loadtxt("X_test.csv", delimiter=',', skiprows=1)
y_train = np.loadtxt("y_train.csv", delimiter=',', skiprows=1)[:,1]

#converting csv into pandas dataframe
#X_train = pd.read_csv('X_train.csv') 
#X_test = pd.read_csv('X_test.csv')
#y_train = pd.read_csv('y_train.csv')

#drop rows are either CHMIN
#row_0_values = (X_train != 0).all(1)
#print (row_0_values)
#X_train = X_train[(X_train != 0).all(1)]
#y_train = y_train[row_0_values]
#print (X_train.size)


#scaling 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(168, 6)
(41, 6)
(168,)


In [2]:
# Arrange answer in two columns. First column (with header "Id") is an
# enumeration from 0 to n-1, where n is the number of test points. Second
# column (with header "EpiOrStroma" is the predictions.
def saveFile(y_pred,name):
    test_header = "Id,PRP"
    n_points = X_test.shape[0]
    y_pred_pp = np.ones((n_points, 2))
    y_pred_pp[:, 0] = range(n_points)
    y_pred_pp[:, 1] = y_pred
    np.savetxt(name, y_pred_pp, fmt='%d,%f', delimiter=",",
               header=test_header, comments="")


In [3]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
scaler = StandardScaler()

#scaling 
X_train1_scaled = scaler.fit_transform(X_train1)
X_test1_scaled = scaler.fit_transform(X_test1)

print(X_train1.shape)
print(X_test1.shape)
print(y_train1.shape)
print(y_test1.shape)

print(X_train1_scaled.shape)
print(X_test1_scaled.shape)

(117, 6)
(51, 6)
(117,)
(51,)
(117, 6)
(51, 6)


In [4]:
# Defining basic untuned regression models

def basicLinearSVR(X_train1, X_test1, y_train1, y_test1):
    svrReg = LinearSVR(max_iter = 9999)
    svrReg.fit(X_train1, y_train1)
    y_pred = svrReg.predict(X_test1)
    print("R2 score: " + str(svrReg.score(X_train1, y_train1)))
    print("Explained variance: " + str(explained_variance_score(y_test1, y_pred)))
    print("Max error: " + str(max_error(y_test1, y_pred)))
    print("Mean absolute error: " + str(mean_absolute_error(y_test1, y_pred)))
    print("Mean squared error: " + str(mean_squared_error(y_test1, y_pred)))
    return y_pred

In [5]:
basicLinearSVR(X_train1_scaled, X_test1_scaled, y_train1, y_test1)

R2 score: 0.4987626610928281
Explained variance: 0.7011141050038857
Max error: 238.94983682344952
Mean absolute error: 32.45855616495046
Mean squared error: 4071.193609463763


array([177.57257888,  26.06481603,  97.7275538 ,  36.51143953,
        14.58071053,  11.67462405,  12.85074128,  14.81806027,
        67.45317143,  22.94882907,  50.24123997,  67.52160701,
        55.32473952,  20.86191261,  31.00980144,  39.23762475,
       111.73405068,  44.9826457 , 250.22523064,  41.96076227,
         8.03876791,  66.24397322,  66.12704935,  19.69583589,
        24.20308772,  95.53338736,   0.34458445,  57.95478516,
       226.05016318,  63.50455474,  20.86191261,  19.8126588 ,
        53.14689029,  86.36813971,  30.0054794 , 188.54504115,
       177.46590645,  36.27324471,  67.110661  ,  85.92076595,
        23.60950049,  47.17609156,  96.81708047,  22.92248754,
       123.03719524,  48.03579627,   8.15906401,  26.47876035,
       281.16989147,  17.63931925,  -2.28411582])

In [6]:
# Linear Support Vector Regression - Further Tuning with Grid Search
lsvr = LinearSVR()
# Look at parameters used by our regression
print('Parameters currently in use:\n')
print(lsvr.get_params())

#Creating the Random Grid
#pipe = Pipeline([('regression model' , LinearSVR())])

param_grid = [
    {#'regression' : [LinearSVR()],
    'epsilon' : [0,75, 0.5, 0.25, 0],
    'tol' : [1e-4, 2e-4],
     'C' : np.linspace(0.5,100,200),
    'fit_intercept' : [True, False],
    #'intercept_scaling' : [0, 0.5, 1, 1.5, 2],
    #'dual' : [True, False],
    'verbose' : [0, 1],
    'max_iter' : [500, 1000, 1500, 2000, 2500, 3000]}
]


Parameters currently in use:

{'C': 1.0, 'dual': True, 'epsilon': 0.0, 'fit_intercept': True, 'intercept_scaling': 1.0, 'loss': 'epsilon_insensitive', 'max_iter': 1000, 'random_state': None, 'tol': 0.0001, 'verbose': 0}


In [297]:
print(np.linspace(0.5,100,200))

[  0.5   1.    1.5   2.    2.5   3.    3.5   4.    4.5   5.    5.5   6.
   6.5   7.    7.5   8.    8.5   9.    9.5  10.   10.5  11.   11.5  12.
  12.5  13.   13.5  14.   14.5  15.   15.5  16.   16.5  17.   17.5  18.
  18.5  19.   19.5  20.   20.5  21.   21.5  22.   22.5  23.   23.5  24.
  24.5  25.   25.5  26.   26.5  27.   27.5  28.   28.5  29.   29.5  30.
  30.5  31.   31.5  32.   32.5  33.   33.5  34.   34.5  35.   35.5  36.
  36.5  37.   37.5  38.   38.5  39.   39.5  40.   40.5  41.   41.5  42.
  42.5  43.   43.5  44.   44.5  45.   45.5  46.   46.5  47.   47.5  48.
  48.5  49.   49.5  50.   50.5  51.   51.5  52.   52.5  53.   53.5  54.
  54.5  55.   55.5  56.   56.5  57.   57.5  58.   58.5  59.   59.5  60.
  60.5  61.   61.5  62.   62.5  63.   63.5  64.   64.5  65.   65.5  66.
  66.5  67.   67.5  68.   68.5  69.   69.5  70.   70.5  71.   71.5  72.
  72.5  73.   73.5  74.   74.5  75.   75.5  76.   76.5  77.   77.5  78.
  78.5  79.   79.5  80.   80.5  81.   81.5  82.   82.5  83.   83

In [280]:
#fit the model with the parameter
reg = GridSearchCV(LinearSVR(), param_grid = param_grid, cv = 10, verbose=True, n_jobs=-1)
# Fit the random search model
best_reg= reg.fit(X_train1_scaled, y_train1)


best_reg.best_params_

Fitting 10 folds for each of 48000 candidates, totalling 480000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 12436 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 35936 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 68836 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done 111136 tasks      | elapsed:   38.8s
[Parallel(n_jobs=-1)]: Done 162836 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 223936 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 294436 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 374336 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 463636 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 480000 out of 480000 | elapsed:  3.6min finished


[LibLinear]

/home/nicholas/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'C': 6.5,
 'epsilon': 0.5,
 'fit_intercept': True,
 'max_iter': 1500,
 'tol': 0.0002,
 'verbose': 1}

In [294]:
# tuned regression models

def tunedLinearSVR(X_train1, X_test1, y_train1, y_test1):
    svrReg = LinearSVR(C= 6.5, epsilon= 0.5, fit_intercept = True, max_iter = 1500, tol = 0.0002, verbose = 1 )
    svrReg.fit(X_train1, y_train1)
    y_pred = svrReg.predict(X_test1)
    print("R2 score: " + str(svrReg.score(X_test1, y_test1)))
    print("Explained variance: " + str(explained_variance_score(y_test1, y_pred)))
    print("Max error: " + str(max_error(y_test1, y_pred)))
    print("Mean absolute error: " + str(mean_absolute_error(y_test1, y_pred)))
    print("Mean squared error: " + str(mean_squared_error(y_test1, y_pred)))
    return y_pred

In [295]:
y_pred = tunedLinearSVR(X_train1_scaled, X_test1_scaled, y_train1, y_test1)


[LibLinear]R2 score: 0.8020073197075291
Explained variance: 0.8100024529525531
Max error: 181.11212399785097
Mean absolute error: 34.448176151688926
Mean squared error: 2459.8676064244405


In [301]:
svrReg = LinearSVR(C= 100, epsilon= 0.5, fit_intercept = True, max_iter = 3000, tol = 0.0002, verbose = 1 )
svrReg.fit(X_train_scaled, y_train)
y_pred = svrReg.predict(X_test_scaled)
saveFile(y_pred,'svr_tuned_scaled.csv')
svrReg.score(X_train_scaled, y_train)

[LibLinear]

0.8474789235030437

In [5]:
#basic model for ridge regression
rigReg = Ridge()
rigReg.fit(X_train1, y_train1)

def basicRidgeReg(X_train1, X_test1, y_train1, y_test1):
    rigReg = Ridge()
    rigReg.fit(X_train1, y_train1)
    y_pred = rigReg.predict(X_test1)
    print("R2 score: " + str(rigReg.score(X_test1, y_test1)))
    print("Explained variance: " + str(explained_variance_score(y_test1, y_pred)))
    print("Max error: " + str(max_error(y_test1, y_pred)))
    print("Mean absolute error: " + str(mean_absolute_error(y_test1, y_pred)))
    print("Mean squared error: " + str(mean_squared_error(y_test1, y_pred)))
    return y_pred

In [6]:
basicRidgeReg(X_train1, X_test1, y_train1, y_test1)

R2 score: 0.7825060598137341
Explained variance: 0.7897274648240552
Max error: 186.00460404912445
Mean absolute error: 35.13322383229487
Mean squared error: 2702.151904138626


array([244.67413306,   3.60428569, 120.32052783,  35.87954244,
        -6.84974768, -14.31132657,  -9.97508057, -14.93014543,
        83.55025995,  28.89142762,  69.74994622,  90.59002238,
        86.38847148,  31.57949605,  24.27283306,  75.07755755,
       125.92698484,  36.02575197, 465.84092056,  34.7669558 ,
        -9.67020154, 113.63825911,  73.2928649 ,   9.80244081,
         5.86985328, 150.25376163,  17.00606619,  60.49107861,
       455.77055124, 112.06271361,  31.57949605,   6.12693791,
        55.73982761, 102.68640076,  41.65242223, 326.00460405,
       304.43478124,  42.88015438, 103.83633654, 120.93277589,
        53.86453507,  56.24675112, 111.49778129,  17.48158777,
       222.16424562,  50.94388459,  -4.92303542,  19.07002806,
       522.6054014 ,   5.98412926,  38.67326368])

In [9]:
# Ridge Regression - Further Tuning with Grid Search
rr = Ridge()
# Look at parameters used by our regression
print('Parameters currently in use:\n')
print(rr.get_params())


#Creating the Random Grid
#pipe = Pipeline([('regression model' , LinearSVR())])

param_grid = [
    {'alpha' : np.linspace(0.5,10,20),
    'copy_X' : [True, False],
    'fit_intercept' : [True, False],
    'max_iter' :  [None, 5, 10, 25, 50],
    'normalize' : [True, False],
    'random_state' : [None, 2,3,4,5,6,7,8,9,10],
    'solver' : ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
    'tol' : [1e-4, 2e-4]}
]

Parameters currently in use:

{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 'tol': 0.001}


In [10]:
#fit the model with the parameter
reg = GridSearchCV(Ridge(), param_grid = param_grid, cv = 10, verbose=True, n_jobs=-1)
# Fit the random search model
best_reg= reg.fit(X_train1_scaled, y_train1)


best_reg.best_params_

Fitting 10 folds for each of 112000 candidates, totalling 1120000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  39 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 12017 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 35017 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done 67217 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-1)]: Done 108617 tasks      | elapsed:   39.6s
[Parallel(n_jobs=-1)]: Done 159217 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 219017 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 288017 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 366217 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 453617 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 550217 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 656017 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 771017 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 895217 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)

{'alpha': 0.5,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': 5,
 'normalize': True,
 'random_state': None,
 'solver': 'sag',
 'tol': 0.0001}

In [32]:
# tuned regression models

def tunedRidgeRegression(X_train1, X_test1, y_train1, y_test1):
    rigReg = Ridge(alpha = 0.5, copy_X = True, fit_intercept = True, max_iter = 100, normalize = True, solver = "sag", tol = 0.0001)
    rigReg.fit(X_train1, y_train1)
    y_pred = rigReg.predict(X_test1)
    print("R2 score: " + str(rigReg.score(X_test1, y_test1)))
    print("Explained variance: " + str(explained_variance_score(y_test1, y_pred)))
    print("Max error: " + str(max_error(y_test1, y_pred)))
    print("Mean absolute error: " + str(mean_absolute_error(y_test1, y_pred)))
    print("Mean squared error: " + str(mean_squared_error(y_test1, y_pred)))
    return y_pred

In [33]:
tunedRidgeRegression(X_train1, X_test1, y_train1, y_test1)

R2 score: 0.8153639880974343
Explained variance: 0.8254982347782042
Max error: 153.79691218913842
Mean absolute error: 33.54332970475656
Mean squared error: 2293.9239167206238


array([254.82544079,  29.19203332, 130.91047709,  49.27320611,
        13.75125597,   7.40768724,  10.34567704,  13.01703533,
        89.4992384 ,  36.65828625,  77.04622625, 100.9334412 ,
        84.23629141,  23.30387509,  41.36673408,  63.63839811,
       158.97804972,  55.49392123, 398.86346731,  52.65821276,
         6.3613055 , 115.25657372,  82.94588406,  22.20617062,
        29.78888203, 143.75671005,   0.96734568,  74.51057558,
       376.17779961, 110.49339061,  23.30387509,  27.37109067,
        73.3331846 , 113.85419125,  50.67486644, 293.79691219,
       267.89358321,  50.4897111 ,  98.23692193, 124.90424752,
        37.65833271,  67.16579082, 131.17758887,  32.84765537,
       198.55286993,  62.2179525 ,   8.29213521,  29.39753338,
       451.85147323,  18.89706049,   2.48446559])

In [34]:
def tunedRidgeRegression2(X_train1, X_test1, y_train1, y_test1):
    rigReg = Ridge(alpha = 1, copy_X = True, fit_intercept = True, normalize = True,random_state = 5, solver = "saga", tol = 0.0002)
    rigReg.fit(X_train1, y_train1)
    y_pred = rigReg.predict(X_test1)
    print("R2 score: " + str(rigReg.score(X_test1, y_test1)))
    print("Explained variance: " + str(explained_variance_score(y_test1, y_pred)))
    print("Max error: " + str(max_error(y_test1, y_pred)))
    print("Mean absolute error: " + str(mean_absolute_error(y_test1, y_pred)))
    print("Mean squared error: " + str(mean_squared_error(y_test1, y_pred)))
    return y_pred

In [35]:
tunedRidgeRegression(X_train1, X_test1, y_train1, y_test1)

R2 score: 0.8155011130039174
Explained variance: 0.8256432098221604
Max error: 153.74948686448943
Mean absolute error: 33.51870396060153
Mean squared error: 2292.2202723485507


array([254.76746596,  29.13318827, 130.85869205,  49.214485  ,
        13.73535246,   7.39941092,  10.33749879,  12.97855464,
        89.44024356,  36.69536149,  77.0208812 , 100.90010541,
        84.17829271,  23.53699765,  41.34179123,  63.71670096,
       158.92048439,  55.45550309, 398.79329716,  52.62022789,
         6.40899761, 115.22699744,  82.88402663,  22.25134572,
        29.77497644, 143.70365922,   1.32004493,  74.44110779,
       376.11109561, 110.44057318,  23.53699765,  27.35889345,
        73.2945535 , 113.79223822,  50.65131079, 293.74948686,
       267.82013088,  50.48178176,  98.17778834, 124.84909015,
        37.93138092,  67.12653861, 131.11499033,  32.83200884,
       198.49526296,  62.14839281,   8.34349228,  29.44336479,
       451.79366633,  18.94374307,   2.99416187])

In [36]:
rigReg = Ridge(alpha = 1, copy_X = True, fit_intercept = True, normalize = True,random_state = 5, solver = "saga", tol = 0.0002)
rigReg.fit(X_train_scaled, y_train)
y_pred = rigReg.predict(X_test_scaled)
saveFile(y_pred,'ridgeReg_tuned_scaled.csv')
rigReg.score(X_train_scaled, y_train)

0.8175773170785894

In [91]:
# Ridge Regression - Further Tuning with Grid Search
gr = GradientBoostingRegressor()
# Look at parameters used by our regression
print('Parameters currently in use:\n')
print(gr.get_params())


#Creating the Random Grid
#pipe = Pipeline([('regression model' , LinearSVR())])

param_grid = [
   {#'alpha': [0.5,0.9,1.5],
    #'criterion': 'friedman_mse', 
    #'init': None, 
    'learning_rate': [0.1, 0.5],
    'loss': ['ls', 'lad', 'huber', 'quantile'], 
    'max_depth': [3, 5],
    #'max_features': None, 
    #'max_leaf_nodes': None, 
    #'min_impurity_decrease': 0.0, 
    #'min_impurity_split': None, 
    #'min_samples_leaf': 1, 
    'min_samples_split': [2, 3, 5],
    #'min_weight_fraction_leaf': 0.0, 
    'n_estimators': [100, 500, 1000],
    #'n_iter_no_change': None, 
    #'presort': 'auto', 
    #'random_state': None, 
    'subsample': [0.5,1.0],
    'tol': [0.0001, 0.0002],
    'validation_fraction': [0.1, 0.5, 1],
    'verbose': [0,1], 
    'warm_start': [True, False]}
]

Parameters currently in use:

{'alpha': 0.9, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'ls', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'auto', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


In [92]:
#fit the model with the parameter
gr = GridSearchCV(GradientBoostingRegressor(), param_grid = param_grid, cv = 10, verbose=True, n_jobs=-1)
# Fit the random search model
best_reg= gr.fit(X_train1_scaled, y_train1)


best_reg.best_params_

Fitting 10 folds for each of 6912 candidates, totalling 69120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 578 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 1050 tasks      | elapsed:   27.4s
[Parallel(n_jobs=-1)]: Done 1400 tasks      | elapsed:   51.2s
[Parallel(n_jobs=-1)]: Done 2465 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 3111 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 4217 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 5453 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 6784 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 8214 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 9264 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 10414 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 11664 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done 13014 tasks      | elapsed: 14.8min
[Parallel(n_jobs=-1)]: Done 14464 tasks  

{'learning_rate': 0.1,
 'loss': 'huber',
 'max_depth': 3,
 'min_samples_split': 5,
 'n_estimators': 500,
 'subsample': 0.5,
 'tol': 0.0001,
 'validation_fraction': 1,
 'verbose': 0,
 'warm_start': False}

In [10]:
# Ridge Regression - Further Tuning with Grid Search
rfr = RandomForestRegressor()
# Look at parameters used by our regression
print('Parameters currently in use:\n')
print(rfr.get_params())


#Creating the Random Grid
#pipe = Pipeline([('regression model' , LinearSVR())])

param_grid = [
   {#'ccp_alpha': [0.0, 0.1, 0.5],
    #'criterion': 'friedman_mse', 
    #'init': None, 
    #'bootstrap' : [True, False],
    'max_depth': [None, 3, 5, 7],
    'oob_score' : [True, False],
    #'max_features': None, 
    #'max_leaf_nodes': None, 
    #'min_impurity_decrease': 0.0, 
    #'min_impurity_split': None, 
    'min_samples_leaf':[1, 2], 
    'min_samples_split': [2, 3, 4],
    #'min_weight_fraction_leaf': 0.0, 
    'n_estimators': [100, 500, 1000, 2500, 5000],
    #'n_iter_no_change': None, 
    #'presort': 'auto', 
    #'random_state': None, 
    #'subsample': [0.5,1.0],
    'verbose': [0,1], 
    #'n_jobs' : -1,
    'warm_start': [True, False]}
]

Parameters currently in use:

{'bootstrap': True, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 'warn', 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [12]:
#fit the model with the parameter
rfr = GridSearchCV(RandomForestRegressor(), param_grid = param_grid, cv = 10, verbose=True, n_jobs=-1)
# Fit the random search model
best_reg= rfr.fit(X_train1_scaled, y_train1)


best_reg.best_params_

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    5.0s finished
/home/nicholas/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'n_estimators': 50, 'oob_score': True, 'warm_start': False}

In [26]:
#basic ensemble regressor

# random_state=1, n_estimators=10
#reg1 = GradientBoostingRegressor(random_state=1, n_estimators=10)
reg2 = RandomForestRegressor(random_state=1, n_estimators=10)
#reg3 = LinearRegression()
#reg4 = Ridge()
reg5 = LinearSVR(max_iter = 100000)
#reg6 = linear_model.Lasso()
#reg7 = LogisticRegression()

ereg = VotingRegressor(estimators=[('rf', reg2),('linearSVR', reg5)])
#ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3),('ridge',reg4)
#                                   ,('lasso', reg6),('logisticR', reg7)])
ereg = ereg.fit(X_train, y_train)
y_pred = ereg.predict(X_test)
print(ereg.score(X_train_scaled, y_train))
saveFile(y_pred,'ensemble_regressor.csv')

-0.19500134705293218


/home/nicholas/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [32]:
#tuned version of ensemble regressor

#reg1 = GradientBoostingRegressor(learning_rate = 0.1, loss = 'huber', max_depth = 3,min_samples_split = 5,n_estimators = 500,
#               subsample = 0.5, tol = 0.0001,validation_fraction = 1, verbose = 0,warm_start = False)
reg2 = RandomForestRegressor(max_depth = None, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 50,
                             oob_score = True, verbose = 0, warm_start = True)
#reg3 = LinearRegression()
#reg4 = Ridge(alpha = 0.5, copy_X = True, fit_intercept = True, max_iter = 100, normalize = True, solver = "sag", tol = 0.0001)
reg5 = LinearSVR(C= 100, epsilon= 0.5, fit_intercept = True, max_iter = 3000, tol = 0.0002, verbose = 1 )
#reg6 = linear_model.Lasso()
#reg7 = LogisticRegression()
#reg8 = SVR(C=1, degree = 1, epsilon = 0, gamma = 1, kernel = "linear", max_iter = 500)


ereg = VotingRegressor(estimators=[('rf', reg2),('linearSVR', reg5)])
ereg = ereg.fit(X_train_scaled, y_train)
y_pred = ereg.predict(X_test_scaled)
print(ereg.score(X_train_scaled, y_train))
saveFile(y_pred,'voting_regressor_.csv')

[LibLinear]0.9370709793306613


In [8]:
gbr = GradientBoostingRegressor(learning_rate = 0.1, loss = 'huber', max_depth = 3,min_samples_split = 5,n_estimators = 500,
               subsample = 0.5, tol = 0.0001,validation_fraction = 1, verbose = 0,warm_start = False)
gbr.fit(X_train_scaled, y_train)
y_pred = gbr.predict(X_test_scaled)
saveFile(y_pred,'gbr_tuned_scaled.csv')
gbr.score(X_train_scaled, y_train)

0.9962720592537098

In [59]:
reg2 = RandomForestRegressor(max_depth = None, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 100,
                             oob_score = True, verbose = 0, warm_start = True)
reg2.fit(X_train_scaled, y_train)
y_pred = reg2.predict(X_test_scaled)
saveFile(y_pred,'randomF_tuned_scaled.csv')
reg2.score(y_train, y_train)

0.9784110592502796

In [ ]:
reg = LinearRegression().fit(X_train1, y_train1)
reg.score(X_train1, y_train1)

In [ ]:
reg = LinearRegression().fit(X_train1, y_train1)
reg.score(X_train1, y_train1)

In [58]:
# k nearest neighbour regresion
neigh = KNeighborsRegressor()
neigh.fit(X_train1, y_train1)
y_pred = neigh.predict(X_test1)
neigh.score(y_test1, y_pred)

ValueError: Expected 2D array, got 1D array instead:
array=[212.  26.  74.  71.  30.  12.  21.  26.  80.  45.  33.  70.  42.  16.
  40.  32. 274.  32. 465.  26.  69.  61.  40.  23.  19. 138.   7.  66.
 465.  64.  12.  22.  33.  65.  35. 140. 172.  50.  46. 113.  37.  62.
  32.  38. 100.  49.  16.  40. 510.  77.  18.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.